In [ ]:
import os
import pandas as pd
import joblib

In [ ]:
data_folder = os.path.join('..','00.data','output')

files_found = [x for x in os.listdir(data_folder) if x.endswith('.pickle')]
files_found_tokens=[x.split('.') for x in files_found]

print(files_found)
print(files_found_tokens)

models_found = {}

for current_model in files_found_tokens:
    model_name = '.'.join(current_model[:-2])
    filename = os.path.join('..','00.data','output','.'.join(current_model))
    if not model_name in models_found:
        models_found[model_name]= {}          
    models_found[model_name][current_model[-2]]=filename

print(models_found)

In [ ]:
first_model = list(models_found.keys())[0]
current_train_dataset = joblib.load(models_found[first_model]['train'])
df_current_train_dataset = pd.DataFrame(current_train_dataset)
df_current_train_dataset.head()
                                                              